# Problem set 2 (45 + 50 + 33 + 15 = 143 pts)

## Problem 1 (LU decomposition) 45 pts


### 1. LU  for band matrices and Cholesky decomposition (13 pts)

The complexity to find an LU decomposition of a dense $n\times n$ matrix is $\mathcal{O}(n^3)$.
Significant reduction in complexity can be achieved if the matrix has a certain structure, e.g. it is sparse. 
In the following task we consider an important example of $LU$ for a special type of matrices –– band matrices with top left entry equal to 1 and the bandwidth $m$ equal to 3 or 5 which called tridiagonal and pentadiagonal respectively. The bands may be ```[1, 2, 1]``` and ```[1, 1, 2, 1, 1]``` respectively

- (4 pts) Write a function ```band_lu(diag_broadcast, n)``` which computes LU decomposition for tridiagonal or pentadiagonal matrix with top left entry equal to 1 with given diagonal bands. 
For example, input parametres ```(diag_broadcast = [1,2,1], n = 4)``` mean that we need to find LU decomposition for the triangular matrix of the form:

$$A = \begin{pmatrix}
1 & 1 & 0 & 0\\
1 & 2 & 1 & 0 \\
0 & 1 & 2 & 1 \\
0 & 0 & 1 & 2 \\
\end{pmatrix}.$$

Provide the extensive testing of the implemented function that will works correctly for large $n$,  e.g. $n=100$.
As an output it is considered to make ```L``` and ```U``` - 2D arrays representing diagonals in factors $L$ (```L[0]``` keeps first lower diagonal, ```L[1]``` keeps second lower, ...), and $U$ (```U[:,0]``` keeps main diagonal, ```U[:,1]``` keeps first upper, ...).

- (2 pts) Compare execution time of the band LU decomposition using standard function from ```scipy```, i.e. which takes the whole matrix and does not know about its special structure, and band decomposition of yours implementation. Comment on the results.

- (7 pts) Write a function ```cholesky(n)``` for computing Cholesky decomposition. It should take the the single argument - the matrix that will be factorized and return the single output - lower-triangular factor $L$. Think about the efficiency of your implementation and if necessary update it to achieve the best performance (eliminate Python loops, where it is possible and so on). Explicitly describe the difference with LU decomposition that reduces the complexity from $2n^3/3$ for LU to $n^3/3$ for Cholesky. 
Test the implemented function on the Pascal matrix of given size $n$ for $n = 5, 10, 50$. 
Pascal matrix is square matrix of the following form (here for $n=4$)
$$P = \begin{pmatrix}
1 & 1 & 1 & 1\\
1 & 2 & 3 & 4 \\
1 & 3 & 6 & 10 \\
1 & 4 & 10 & 20 \\
\end{pmatrix}.$$

    [Here](https://en.wikipedia.org/wiki/Pascal_matrix) you can find more details about such matrices and analytical form for factor $L$ from Cholesky decomposition. Compare the result of your implementation with analytical expression in terms of some matrix norm of difference.  

In [76]:
from scipy.sparse import diags # can be used with broadcasting of scalars if desired dimensions are large
import numpy as np

from math import sqrt

# INPUT : diag_broadcast - list of diagonals value to broadcast,length equal to 3 or 5; n - integer, band matrix shape.
# OUTPUT : L - 2D np.ndarray, L.shape[0] depends on bandwidth, L.shape[1] = n-1, do not store main diagonal, where all ones;
#          add zeros to the right side of rows to handle with changing length of diagonals.
#          U - 2D np.ndarray, U.shape[0] = n, U.shape[1] depends on bandwidth;
#          add zeros to the bottom of columns to handle with changing length of diagonals.
def band_lu(d_b, n): #d_b is a diag_broadcast
  m = len(d_b)
  if(m==3):
    L=np.zeros((1,n-1))
    U=np.zeros((n,2))
    U[0,0]=1
    L[0,0]=d_b[0]
    for i in range(1,n-1):
      L[0,i]=d_b[0]/(d_b[1]-d_b[2]*L[0,i-1])
      U[i,0]=d_b[1]-L[0,i-1]*d_b[2]
      U[i-1,1]=d_b[2]
    U[-1,0]=d_b[1]-L[0,-1]*d_b[2]
    U[-2,1]=d_b[2]

  elif(m==5):
    L=np.zeros((2,n-1))
    U=np.zeros((n,3))
    U[0,0]=1
    U[0,1]=d_b[3]
    U[0,2]=d_b[4]
    
    L[0,0]=d_b[1]/U[0,0]
    U[1,0]=d_b[2]-L[0,0]*U[0,1]
    U[1,1]=d_b[3]-L[0,0]*U[0,2]
    U[1,2]=d_b[4]

    for i in range(0,n-2):
      L[1,i]=d_b[0]/U[i,0]
      L[0,i+1]=( d_b[1]-L[1,i]*U[i,1] )/U[i+1,0]
      
      U[i+2,0]=d_b[2]-L[1,i]*U[i,2]-L[0,i+1]*U[1+i,1]
      U[i+2,1]=d_b[3]-L[0,i+1]*U[i+1,2]
      U[i+2,2]=d_b[4]
    U[-2,2]=0
    U[-1,2]=0
    U[-1,1]=0
    L[1,-1]=0
  return L,U
    
def cholesky(A):
    n = len(A)

    L = np.zeros((n,n))

    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i,j] * L[k,j] for j in range(k))
            
            if (i == k): # Diagonal elements
                if(A[i][i] - tmp_sum < 0):
                  print(f"We need to take a sqrt of the negative number of  A[i][i]-tmp_sum for A[i][i]= { A[i][i] } and tmp_sum = {tmp_sum}\n")
                L[i,k] =sqrt(A[i][i] - tmp_sum)
            else:
                L[i,k] = (1.0 / L[k,k] * (A[i][k] - tmp_sum))
    return L

1. band_lu

In [ ]:
L,U=band_lu([4,2,7],5)

In [ ]:
L

array([[ 4.        , -0.15384615,  1.3       , -0.56338028]])

In [ ]:
U

array([[  1.        ,   7.        ],
       [-26.        ,   7.        ],
       [  3.07692308,   7.        ],
       [ -7.1       ,   7.        ],
       [  5.94366197,   0.        ]])

In [ ]:
L,U=band_lu([4,2,7,8,9],5)

In [ ]:
L

array([[  2.        ,   3.33333333,  -0.56410256, -15.81818182],
       [  4.        ,  -0.44444444,   0.92307692,   0.        ]])

In [ ]:
U

array([[  1.        ,   8.        ,   9.        ],
       [ -9.        , -10.        ,   9.        ],
       [  4.33333333, -22.        ,   9.        ],
       [ -1.41025641,  13.07692308,   0.        ],
       [205.54545455,   0.        ,   0.        ]])

2. Time comparison

In [ ]:
from scipy.linalg import lu
n=1000
A=np.zeros((n,n))
diag_broadcast = [4,2,7]
A[0,0]=1
A[0,1]=diag_broadcast[2]
for i in range(n-2):
  A[i+1,i:(i+3)]=diag_broadcast
A[-1,-2:]=diag_broadcast[:-1]

In [ ]:
%%timeit
p , l , u = lu(A)

10 loops, best of 5: 42.5 ms per loop


In [ ]:
%%timeit
L,U=band_lu([4,2,7],1000)

100 loops, best of 5: 2.61 ms per loop


In [ ]:
%%timeit
L,U=band_lu([4,2,7,8,9],1000)

100 loops, best of 5: 4.9 ms per loop


3.Cholesky decomposition

In [77]:
from scipy.linalg import pascal
for i in [5,10,50]:
  A = pascal(i)
  L = cholesky(A)
  print(f"n = {i}\n")
  print("A:\n",np.array(A))
  print("L:\n",L)

n = 5

A:
 [[ 1  1  1  1  1]
 [ 1  2  3  4  5]
 [ 1  3  6 10 15]
 [ 1  4 10 20 35]
 [ 1  5 15 35 70]]
L:
 [[1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 2. 1. 0. 0.]
 [1. 3. 3. 1. 0.]
 [1. 4. 6. 4. 1.]]
n = 10

A:
 [[    1     1     1     1     1     1     1     1     1     1]
 [    1     2     3     4     5     6     7     8     9    10]
 [    1     3     6    10    15    21    28    36    45    55]
 [    1     4    10    20    35    56    84   120   165   220]
 [    1     5    15    35    70   126   210   330   495   715]
 [    1     6    21    56   126   252   462   792  1287  2002]
 [    1     7    28    84   210   462   924  1716  3003  5005]
 [    1     8    36   120   330   792  1716  3432  6435 11440]
 [    1     9    45   165   495  1287  3003  6435 12870 24310]
 [    1    10    55   220   715  2002  5005 11440 24310 48620]]
L:
 [[  1.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   1.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   2.   1.   0.   0.   0.   0.   0.   0.   0

ValueError: ignored

### 2. Stability of LU (8 pts)

* (4 pts) Show, that for these  matrices $A$ and $B$ LU decomposition fails. Why does it happen?



$
A = \begin{pmatrix}
0 & 1 \\
2 & 3
\end{pmatrix}.$ 

$B = \begin{pmatrix}
1 & 1 & 0\\
1 & 1 & 2 \\
1 & 2 & 1
\end{pmatrix}.$ 

* (4 pts) In the LU decomposition, a pivot position is a position of the element that identifies the row and column that will be eliminated in the current step. For example, first pivot in LU is usually the left top element. What value of $c$ leads to zero in the second pivot position? What $c$ produces zero in the third pivot position? What modification of LU should we use in order to address the possible zeros in pivot position?

$A = \begin{pmatrix}
1 & c & 0\\
2 & 4 & 1 \\
3 & 5 & 1
\end{pmatrix}.$ 

##### * (4 pts) Show, that for these  matrices $A$ and $B$ LU decomposition fails. Why does it happen?



$
A = \begin{pmatrix}
0 & 1 \\
2 & 3
\end{pmatrix}.$ 

$B = \begin{pmatrix}
1 & 1 & 0\\
1 & 1 & 2 \\
1 & 2 & 1
\end{pmatrix}.$ 


$
A = 
\begin{pmatrix}
0 & 1 \\
2 & 3
\end{pmatrix}
=
\begin{pmatrix}
1 & 0 \\
l & 1
\end{pmatrix}
\cdot
\begin{pmatrix}
u_{11} & u_{12} \\
0 & u_{21}
\end{pmatrix}
=
\begin{pmatrix}
u_{11} & u_{12} \\
l·u_{11} & l·u_{12}+u_{21}
\end{pmatrix}
=|u_{11}=0,u_{12}=1|
=
\begin{pmatrix}
0 & 1 \\
l·0 & l·1+u_{21}
\end{pmatrix}
$ 
$l=2/0 \Rightarrow $decomposition fails



$B =
\begin{pmatrix}
1 & 1 & 0\\
1 & 1 & 2 \\
1 & 2 & 1
\end{pmatrix}
=
\begin{pmatrix}
1 & 0 & 0\\
l_1 & 1 & 0 \\
l_3 & l_2 & 1
\end{pmatrix}
\cdot
\begin{pmatrix}
u_{11} & u_{12} & u_{13}\\
0 & u_{21} & u_{22} \\
0 & 0 & u_{31}
\end{pmatrix}
=
\begin{pmatrix}
u_{11} & u_{12} & u_{13}\\
l_1 \cdot u_{11} & l_1 \cdot u_{12}+u_{21} & l_1 \cdot u_{13}+u_{22} \\
l_3 \cdot u_{11} & l_3 \cdot u_{12}+l_2 \cdot u_{21} & l_3 \cdot u_{13}+l_2 \cdot u_{22} + u_{31}
\end{pmatrix}
=|u_{11}=1,u_{12}=1,u_{13}=0|
=
\begin{pmatrix}
1 & 1 & 0\\
l_1 \cdot 1 & l_1 \cdot 1+u_{21} & l_1 \cdot 0+u_{22} \\
l_3 \cdot 1 & l_3 \cdot 1+l_2 \cdot u_{21} & l_3 \cdot 0+l_2 \cdot u_{22} + u_{31}
\end{pmatrix}
$
Then
$l_1 = 1, u_{21}=0, u_{22}=2, l_3=1,l_2=(2-1)/0 \Rightarrow$ decomposition fails


decomposition fails because during decomposition we need to divide by zero: these matrixes are not strictly regular

##### (4 pts) In the LU decomposition, a pivot position is a position of the element that identifies the row and column that will be eliminated in the current step. For example, first pivot in LU is usually the left top element. What value of $c$ leads to zero in the second pivot position? What $c$ produces zero in the third pivot position? What modification of LU should we use in order to address the possible zeros in pivot position?

$A = \begin{pmatrix}
1 & c & 0\\
2 & 4 & 1 \\
3 & 5 & 1
\end{pmatrix}.$ 

if $4-\dfrac{2}{1}\cdot c = 0$, then $c=2$ leads to zero in second pivot position\
$A = 
\begin{pmatrix}
1 & c & 0\\
2 & 4 & 1 \\
3 & 5 & 1
\end{pmatrix}
=
\begin{pmatrix}
1 & 0 & 0\\
l_1 & 1 & 0 \\
l_3 & l_2 & 1
\end{pmatrix}
\cdot
\begin{pmatrix}
u_{11} & u_{12} & u_{13}\\
0 & u_{21} & u_{22} \\
0 & 0 & u_{31}
\end{pmatrix}
=|\text{zero in the third pivot position} \Rightarrow u_{31}=0|
=
\begin{pmatrix}
u_{11} & u_{12} & u_{13}\\
l_1 \cdot u_{11} & l_1 \cdot u_{12}+u_{21} & l_1 \cdot u_{13}+u_{22} \\
l_3 \cdot u_{11} & l_3 \cdot u_{12}+l_2 \cdot u_{21} & l_3 \cdot u_{13}+l_2 \cdot u_{22}
\end{pmatrix}
=|u_{11}=1,u_{12}=c,u_{13}=0|
=
\begin{pmatrix}
1 & c & 0\\
l_1 \cdot 1 & l_1 \cdot c+u_{21} & l_1 \cdot 0+u_{22} \\
l_3 \cdot 1 & l_3 \cdot c+l_2 \cdot u_{21} & l_3 \cdot 0+l_2 \cdot u_{22}
\end{pmatrix}
=|l_1=2,l_3=3|
=
\begin{pmatrix}
1 & c & 0\\
2 & 2\cdot c+u_{21} & u_{22} \\
3 & 3\cdot c+l_2 \cdot u_{21} & l_2 \cdot u_{22}
\end{pmatrix}
=|u_{22}=1|
=
\begin{pmatrix}
1 & c & 0\\
2 & 2\cdot c+u_{21} & 1 \\
3 & 3\cdot c+l_2 \cdot u_{21} & l_2
\end{pmatrix}
=|l_2=1|
=
\begin{pmatrix}
1 & c & 0\\
2 & 2\cdot c+u_{21} & 1 \\
3 & 3\cdot c+u_{21} & 1
\end{pmatrix}
$

$
\left\{\begin{matrix}
2\cdot c + u_{21} &= 4  \\
3\cdot c + u_{21} &= 5  \\
\end{matrix}\right.\Rightarrow c=1 \text{ leads to zero in third pivot position }
$

We need to use PLU decomposition in order to address the possible zeros in pivot position

### 3. Implementation of PLU decomposition (14 pts)

As you have noticed before, LU decomposition may fail. In order to make it stable, we can use LU decomposition with pivoting  (PLU).

We want to find such permutation matrix $P$ that LU decomposition of $PA$ exists

$$ PA = LU $$

- (7 pts) Implement efficiently PLU decomposition (without loops and with appropriate level of BLAS operations).  Also, pay attention to the way of permutation matrix storage.

- (4 pts ) Compare your function for computing PLU with built-in function on matrices of such type ```(mirror_diag = [1,2,1], n = 4)```. (Bandwidth and matrix size may vary). So, you can pass them as dense 2D NumPy array and do not tune your implementation to this special structure. Compare them in terms of running time (use ```%timeit``` magic) for range of dimensions to recover the asymptotic rate of time increasing and in terms of acuracy. We expect you plot the running time vs matrix dimension for built-in function and your implementation. So you should get the plot with two lines.
Consider additionally one of the pathological examples from above, where LU fails, but PLU has to work.


$$A = \begin{pmatrix}
0 & 0 & 1 & 1 \\
 0 &1 & 2 & 1  \\
 1 & 2 & 1  & 0\\
1 & 2  & 0 & 0  \\
\end{pmatrix}.$$


- (3 pts) Discuss the obtained results and explain how is it possible to accelerate computing the PLU factorization. 

NumPy or JAX are both ok in this problem, but please use the single library for all implementations. 

##### - (7 pts) Implement efficiently PLU decomposition (without loops and with appropriate level of BLAS operations).  Also, pay attention to the way of permutation matrix storage.

In [ ]:
import numpy as np

def pivot_matrix(M):
    """Returns the pivoting matrix for M, used in Doolittle's method."""
    m = len(M)

    id_mat = np.identity(m, dtype='float')

    # Rearrange the identity matrix such that the largest element of                                                                                                                                                                                   
    # each column of M is placed on the diagonal of of M                                                                                                                                                                                               
    for j in range(m):
        row = max(range(j, m), key=lambda i: abs(M[i][j]))
        if j != row:
            # Swap the rows                                                                                                                                                                                                                            
            id_mat[j], id_mat[row] = id_mat[row], id_mat[j]

    return id_mat

def lu_decomposition(A):
    """Performs an LU Decomposition of A (which must be square)                                                                                                                                                                                        
    into PA = LU. The function returns P, L and U."""
    A=np.array(A)
    n = len(A)

    # Create zero matrices for L and U                                                                                                                                                                                                                 
    L = np.zeros((n,n))
    U = np.zeros((n,n))

    # Create the pivot matrix P and the multipled matrix PA                                                                                                                                                                                            
    P = pivot_matrix(A)
    PA = np.matmul(P, A)

    # Perform the LU Decomposition                                                                                                                                                                                                                     
    for j in range(n):
        # All diagonal entries of L are set to unity                                                                                                                                                                                                   
        L[j,j] = 1.0

        # LaTeX: u_{ij} = a_{ij} - \sum_{k=1}^{i-1} u_{kj} l_{ik}                                                                                                                                                                                      
        for i in range(j+1):
            s1 = sum(U[k,j] * L[i,k] for k in range(i))
            U[i,j] = PA[i,j] - s1

        # LaTeX: l_{ij} = \frac{1}{u_{jj}} (a_{ij} - \sum_{k=1}^{j-1} u_{kj} l_{ik} )                                                                                                                                                                  
        for i in range(j, n):
            s2 = sum(U[k,j] * L[i,k] for k in range(j))
            L[i,j] = (PA[i,j] - s2) / U[j,j]

    return (P, L, U)

In [ ]:
A = [[7, 3, -1, 2], [3, 8, 1, -4], [-1, 1, 4, -1], [2, -4, -1, 6]]
P, L, U = lu_decomposition(A)

print("A:\n",A)
print("P:\n",P)
print("PA:\n",P@A)
print("L:\n",L)
print("U:\n",U)
print("LU:\n",L@U)
print("PA-LU:\n",P@A-L@U)

A:
 [[7, 3, -1, 2], [3, 8, 1, -4], [-1, 1, 4, -1], [2, -4, -1, 6]]
P:
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
PA:
 [[ 7.  3. -1.  2.]
 [ 3.  8.  1. -4.]
 [-1.  1.  4. -1.]
 [ 2. -4. -1.  6.]]
L:
 [[ 1.          0.          0.          0.        ]
 [ 0.42857143  1.          0.          0.        ]
 [-0.14285714  0.21276596  1.          0.        ]
 [ 0.28571429 -0.72340426  0.08982036  1.        ]]
U:
 [[ 7.          3.         -1.          2.        ]
 [ 0.          6.71428571  1.42857143 -4.85714286]
 [ 0.          0.          3.55319149  0.31914894]
 [ 0.          0.          0.          1.88622754]]
LU:
 [[ 7.  3. -1.  2.]
 [ 3.  8.  1. -4.]
 [-1.  1.  4. -1.]
 [ 2. -4. -1.  6.]]
PA-LU:
 [[ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 -4.4408921e-16]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -4.4408921e-16  0.0000000e+00  0.0000000e+00]]


In [ ]:
from scipy.linalg import lu
p,l,u=lu(A)
print("p:\n",p)
print("pA:\n",p@A)
print("l:\n",l)
print("u:\n",u)
print("lu:\n",l@u)
print("pA-lu:\n",p@A-l@u)

p:
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
PA:
 [[ 7.  3. -1.  2.]
 [ 3.  8.  1. -4.]
 [-1.  1.  4. -1.]
 [ 2. -4. -1.  6.]]
l:
 [[ 1.          0.          0.          0.        ]
 [ 0.42857143  1.          0.          0.        ]
 [-0.14285714  0.21276596  1.          0.        ]
 [ 0.28571429 -0.72340426  0.08982036  1.        ]]
u:
 [[ 7.          3.         -1.          2.        ]
 [ 0.          6.71428571  1.42857143 -4.85714286]
 [ 0.          0.          3.55319149  0.31914894]
 [ 0.          0.          0.          1.88622754]]
lu:
 [[ 7.  3. -1.  2.]
 [ 3.  8.  1. -4.]
 [-1.  1.  4. -1.]
 [ 2. -4. -1.  6.]]
pa-lu:
 [[ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 -4.4408921e-16]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -4.4408921e-16  0.0000000e+00  0.0000000e+00]]


In [ ]:
# Your solution is here

### 4. Block LU (10 pts)

Let $A = \begin{bmatrix} A_{11} & A_{12} \\ A_{21} & A_{22} \end{bmatrix}$ be a block matrix. The goal is to solve the linear system

$$
     \begin{bmatrix} A_{11} & A_{12} \\ A_{21} & A_{22} \end{bmatrix} \begin{bmatrix} u_1 \\ u_2 \end{bmatrix} = \begin{bmatrix} f_1 \\ f_2 \end{bmatrix}.
$$

* (2 pts) Using block elimination find matrix $S$ and right-hand side $\hat{f_2}$ so that $u_2$ can be found from $S u_2 = \hat{f_2}$. Note that the matrix $S$ is called <span style="color:red">Schur complement</span> of the block $A_{11}$.
* (4 pts) Using Schur complement properties prove that 

$$\det(X+AB) = \det(X)\det(I+BX^{-1}A), $$


where $X$ - nonsingular square matrix.
* (4 pts) Let matrix $F \in \mathbb{R}^{m \times n}$ and $G \in \mathbb{R}^{n \times m}$. Prove that 

$$\det(I_m - FG) = \det(I_n - GF).$$

$
\begin{bmatrix}
A_{11} & A_{12} \\ 
A_{21} & A_{22} 
\end{bmatrix}
=
\begin{bmatrix}
L_{11} & 0 \\ 
L_{21} & L_{22} 
\end{bmatrix}
\begin{bmatrix}
U_{11} & U_{12} \\ 
0      & U_{22} 
\end{bmatrix}
=
\begin{bmatrix}
L_{11}U_{11} & L_{11}U_{12} \\ 
L_{21}U_{11} & L_{22}U_{22}+L_{21}U_{12} 
\end{bmatrix}
$

Gaussian elimination:
1.  Factor $A_{11}=L_{11}U_{11}$.
2.  Compute $L_{21}=A_{21}U^{−1}_{11}$ and $U_{12}=L^{−1}_{11}A_{12}$
3.  Form the Schur complement $S=A_{22} − L_{21}U_{12}=L_{22}U_{22}=A_{22}-A_{21}A_{22}^{-1}A_{12}$

## Problem 2 (eigenvalues)  (50 pts)

### 1. Theoretical tasks (15 pts)

* (2 pts) Prove that eigenvectors that correspond to distinct eigenvalues are linearly independent.

* (3 pts) $A$ is a matrix such that $a_{i,j} \ge 0$ and $\sum_{j}a_{i,j} = 1$ (sum of the elements in each row is 1). Prove that $A$ has an eigenvalue $λ=1$ and that any eigenvalue $λ_i$: $|λ_i| \le 1$.

* (5 pts) Prove that normal matrix is Hermitian iff its eigenvalues are real. Prove that normal matrix is unitary iff its eigenvalues satisfy $|λ| = 1$. 

* (5 pts) The following problem illustrates instability of the Jordan form. Find theoretically the eigenvalues of the perturbed Jordan block (there is only one $ɛ$ - in the left lower corner):

$$
    J(ɛ) = 
    \begin{bmatrix} 
     λ & 1 & & & 0 \\ 
     0 & λ & 1 & & \\ 
     & 0 & \ddots & \ddots & \\ 
     & & 0 & λ & 1 \\ 
     ɛ & & & 0 & λ  \\ 
    \end{bmatrix}_{n\times n}
$$

   Comment how eigenvalues of $J(0)$ are perturbed for large $n$.

##### (2 pts) Prove that eigenvectors that correspond to distinct eigenvalues are linearly independent.

Let $A$ be a matrix. Let $[v_1,v_2,…,v_n]$ be eigenvectors corresponding to distinct eigenvalues $[λ_1,λ_2,…,λ_n]$. Then $[v_1,v_2,…,v_n]$ are linearly independent.

**Proof by induction**

Let $S_k=[v_1,v_2,…,v_k]$.

Let $S_k$ is linearly independent if $\sum_{i=1}^{k}a_iv_i=\Theta \Leftrightarrow a_i=\Theta \text{ for } i \in [1,k]$

**Base case:**

1. An empty set is linearly independent by definition. Therefore, $S_0$ is linearly independent. 
2. Since eigenvectors are non-zero, $S_1$ is linearly independent.

**Inductive step:**

Assume that $S_k$ is independent for $1≤k≤n$.

Let $\sum_{i=1}^{k+1}a_iv_i=\Theta$.

$A\Theta=\Theta$

$\Theta=A\Theta=A \left( \sum_{i=1}^{k+1}a_iv_i \right)=\sum_{i=1}^{k+1}a_iA(v_i)=\sum_{i=1}^{k+1}a_iλ_iv_i=a_{k+1}λ_{k+1}v_{k+1}+\sum_{i=1}^{k}a_iλ_iv_i$

$\Theta=λ_{k+1}\Theta=λ_{k+1}\left( \sum_{i=1}^{k+1}a_iv_i \right)=\sum_{i=1}^{k+1}a_iλ_{k+1}v_i=a_{k+1}λ_{k+1}v_{k+1}+\sum_{i=1}^{k}a_iλ_{k+1}v_i$

Subtracting the above 2 equations, we get:

$\Theta=\sum_{i=1}^k a_i(λ_i−λ_{k+1})v_i$

Since $S_k$ is linearly independent, $∀i≤k,a_i(λ_i−λ_{k+1})=0$. Since all $λ_i$ are distinct, $∀i≤k,a_i=0$.

$\Theta=\sum_{i=1}^{k+1}a_iv_i \Rightarrow a_{k+1}v_{k+1}=\Theta-\Theta=\Theta$

Since $v_{k+1}≠\Theta$ (because eigenvectors are non-zero), $a_{k+1}=0$.

Since $∀i≤k+1 \ a_i=0,\text{ then } S_{k+1}$ is linearly independent.

By the principle of mathematical induction, $S_n$ is linearly independent.

##### (3 pts) $A$ is a matrix such that $a_{i,j} \ge 0$ and $\sum_{j}a_{i,j} = 1$ (sum of the elements in each row is 1). Prove that $A$ has an eigenvalue $λ=1$ and that any eigenvalue $λ_i$: $|λ_i| \le 1$.

Let $ v =\begin{bmatrix} 
     1 & 1 & ...& 1 \\ 
    \end{bmatrix}_{1\times n}
$
then
$
    A\cdot v = 
    \begin{bmatrix} 
     a_{11} & a_{12} & ...& a_{1n} \\ 
     a_{21} & a_{22} & ...& a_{2n} \\ 
     ...    & ...    & ...& ...    \\   
     a_{n1} & a_{n2} & ...& a_{nn} \\
    \end{bmatrix}_{n\times n}
    \cdot
    \begin{bmatrix} 
     1 \\
     1 \\
     ... \\
     1 \\     
    \end{bmatrix}_{n \times 1}
    =
    \begin{bmatrix} 
     a_{11} + a_{12} + ...+ a_{1n} \\ 
     a_{21} + a_{22} + ...+ a_{2n} \\ 
     ...    + ...    + ...+ ...    \\   
     a_{n1} + a_{n2} + ...+ a_{nn} \\
    \end{bmatrix}_{n\times 1}
    =
    \begin{bmatrix} 
     1 \\
     1 \\
     ... \\
     1 \\ 
    \end{bmatrix}_{n \times 1}
    = 1 \cdot v \Rightarrow λ = 1 \text{ corresponds to the }  v
$


Let $|v|_{max}=\max_i|v_i|$\
Since $Av=λv$, $\sum_j a_{ij}v_j = λv_i $\
$|λ||v_i|=|\sum_j a_{ij}v_j| \leq \sum_j a_{ij}|v_j| \leq \sum_j a_{ij}|v|_{max} =|v|_{max} \sum_j a_{ij}=|v|_{max}$ for every $v_i \Rightarrow |λ||v|_{max}\leq |v|_{max}, |λ| \leq 1$, **Q.E.D.**

##### (5 pts) Prove that normal matrix is Hermitian if its eigenvalues are real. Prove that normal matrix is unitary if its eigenvalues satisfy $|λ| = 1$. 

Normal matrix: $AA^* = A^*A$, Hermitian matrix: $A=A^*$

- $Av=λv \Rightarrow v^*Av=v^*λv=λ\|v\|^2_2$
- $Av=λv \Rightarrow v^*A^*=(Av)^*=(λv)^*=λv^* \Rightarrow v^*A^*v=λv^*v=λ\|v\|^2_2=v^*Av \Rightarrow A^*=A$, **Q.E.D.**


Unitary matrix: $A^*A=AA^*=I_n$

- $Av=λv$
- $v^*A^*=(Av)^*=(λv)^*=λ^*v^*$
- $v^*A^* \cdot Av = λ^*v^* \cdot λv = v^*v \Rightarrow v^*A^*A = v^* \Rightarrow A^*A=I_n$
- A is the normal matrix, then $AA^*=A^*A=I_n$, **Q.E.D.**

##### (5 pts) The following problem illustrates instability of the Jordan form. Find theoretically the eigenvalues of the perturbed Jordan block (there is only one $ɛ$ - in the left lower corner):

$$
    J(ɛ) = 
    \begin{bmatrix} 
     λ & 1 & & & 0 \\ 
     0 & λ & 1 & & \\ 
     & 0 & \ddots & \ddots & \\ 
     & & 0 & λ & 1 \\ 
     ɛ & & & 0 & λ  \\ 
    \end{bmatrix}_{n\times n}
$$
Comment how eigenvalues of $J(0)$ are perturbed for large $n$.

$
    \det (J(ɛ)-λ_iI_n) = 
    \begin{vmatrix} 
     λ-λ_i & 1 & & & 0 \\ 
     0 & λ-λ_i & 1 & & \\ 
     & 0 & \ddots & \ddots & \\ 
     & & 0 & λ-λ_i & 1 \\ 
     ɛ & & & 0 & λ-λ_i  \\ 
    \end{vmatrix}_{n\times n}
    =
    (λ-λ_i)^n +
    ɛ
    \begin{vmatrix} 
     1 & 0& & 0 \\ 
     λ-λ_i & 1 & 0& \\ 
     0 & \ddots & \ddots & 0\\ 
     & 0 & λ-λ_i & 1 \\ 
    \end{vmatrix}_{(n-1)\times (n-1)}
    =
    (λ-λ_i)^n +
    ɛ
    \begin{vmatrix} 
     1 & λ-λ_i& & 0 \\ 
     0 & 1 & λ-λ_i & \\ 
     0 & \ddots & \ddots & λ-λ_i\\ 
     & 0 & 0 & 1 \\ 
    \end{vmatrix}_{(n-1)\times (n-1)}
    =
    (λ-λ_i)^n +
    ɛ \cdot 1 = 0 \Rightarrow λ_i = λ-\sqrt[n]{-ɛ} 
$

$ ɛ=ɛ_0 · e^{i(α+2πk)} $, $ k \in ℤ $

$ -ɛ=ɛ_0 · e^{i(α+π+2πk)} $, $ k \in ℤ $

$ \sqrt[n]{-ɛ}=\sqrt[n]{ɛ_0} · e^{i(α+π+2πk)/n} $, $ k \in ℤ $\
$λ_i = λ-\sqrt[n]{-ɛ}$ will be evenly distributed in circle around $λ$ 

### 2. PageRank (35 pts)


#### Damping factor importance

* (5 pts) Write the function ```pagerank_matrix(G)``` that takes an adjacency matrix $G$ (in both sparse and dense formats) as an input and outputs the corresponding PageRank matrix $A$.

In [62]:
import numpy as np
from numpy import linalg as LA
# INPUT:  G - np.ndarray or sparse matrix
# OUTPUT: A - np.ndarray (of size G.shape) or sparse matrix
def pagerank_matrix_dense(G):
  n = len(G[0])
  for i in range(n):
    n_neighbors=sum(G[:,i])
    G[:,i]=np.divide(G[:,i],n_neighbors)
  A=np.empty(n)
  A.fill(1.0/n)
  A_temp=np.empty(n)
  while(1):
    A_temp = np.matmul(G,A)
    if(LA.norm(A_temp-A)<1e-7):
      break
    A=A_temp.copy()
  
  return A

In [63]:
pagerank_matrix_dense(np.array([[0,0,1,0],[1,0,1,0],[1,0,0,1],[0,1,1,0]],dtype='float' ) )

array([0.12499998, 0.1875    , 0.37500004, 0.31249999])

* (3 pts) Find PageRank matrix $A$ that corresponds to the following graph: <img src="https://github.com/oseledets/nla2021/blob/master/hw/hw2/graph.png?raw=1" width='250'>
What is its largest eigenvalue? What multiplicity does it have?


* (5 pts) Implement the power method for a given matrix $A$, an initial guess $x_0$ and a number of iterations ```num_iter```. It should be organized as a function ```power_method(A, x0, num_iter)``` that outputs approximation to eigenvector $x$, eigenvalue $λ$ and history of residuals $\{\|Ax_k - λ_k x_k\|_2\}$. Make sure that the method converges to the correct solution on a matrix $\begin{bmatrix} 2 & -1 \\ -1 & 2 \end{bmatrix}$ which is known to have the largest eigenvalue equal to $3$.

In [ ]:
# INPUT:  A - np.ndarray (2D), x0 - np.ndarray (1D), num_iter - integer (positive)
# OUTPUT: x - np.ndarray (of size x0), l - float, res - np.ndarray (of size num_iter + 1 [include initial guess])
def power_method(A, x0, num_iter): # 5 pts
    # enter your code here
    return x, l, res

* (2 pts) Run the power method for the graph presented above and plot residuals $\|Ax_k - λ_k x_k\|_2$ as a function of $k$ for ```num_iter=100``` and random initial guess ```x0```.  Explain the absence of convergence. 


* (2 pts) Consider the same graph, but with additional self loop at node 4 (self loop is an edge that connects a vertex with itself). Plot residuals as in the previous task and discuss the convergence. Now, run the power method with ```num_iter=100``` for 10 different initial guesses and print/plot the resulting approximated eigenvectors. Why do they depend on the initial guess?


In order to avoid this problem Larry Page and Sergey Brin [proposed](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf) to use the following regularization technique:

$$
A_d = dA + \frac{1-d}{N} \begin{pmatrix} 1 & \dots & 1 \\ \vdots & & \vdots \\ 1 & \dots & 1 \end{pmatrix},
$$

where $d$ is a small parameter in $[0,1]$ (typically $d=0.85$), which is called **damping factor**, $A$ is of size $N\times N$. Now $A_d$ is the matrix with multiplicity of the largest eigenvalue equal to 1. 
Recall that computing the eigenvector of the PageRank matrix, which corresponds to the largest eigenvalue, has the following interpretation. Consider a person who stays in a random node of a graph (i.e. opens a random web page); at each step s/he follows one of the outcoming edges uniformly at random (i.e. opens one of the links). So the person randomly walks through the graph and the eigenvector we are looking for is exactly his/her stationary distribution â€” for each node it tells you the probability of visiting this particular node. Therefore, if the person has started from a part of the graph which is not connected with the other part, he will never get there.  In the regularized model, the person at each step follows one of the outcoming links with probability $d$ OR teleports to a random node from the whole graph with probability $(1-d)$.

* (2 pts) Now, run the power method with $A_d$ and plot residuals $\|A_d x_k - λ_k x_k\|_2$ as a function of $k$ for $d=0.97$, ```num_iter=100``` and a random initial guess ```x0```.

* (5 pts) Find the second largest in the absolute value eigenvalue of the obtained matrix $A_d$. How and why is it connected to the damping factor $d$? What is the convergence rate of the PageRank algorithm when using damping factor?

Usually, graphs that arise in various areas are sparse (social, web, road networks, etc.) and, thus, computation of a matrix-vector product for corresponding PageRank matrix $A$ is much cheaper than $\mathcal{O}(N^2)$. However, if $A_d$ is calculated directly, it becomes dense and, therefore, $\mathcal{O}(N^2)$ cost grows prohibitively large for  big $N$.


* (2 pts) Implement fast matrix-vector product for $A_d$ as a function ```pagerank_matvec(A, d, x)```, which takes a PageRank matrix $A$ (in sparse format, e.g., ```csr_matrix```), damping factor $d$ and a vector $x$ as an input and returns $A_dx$ as an output. 

* (1 pts) Generate a random adjacency matrix of size $10000 \times 10000$ with only 100 non-zero elements and compare ```pagerank_matvec``` performance with direct evaluation of $A_dx$.

In [ ]:
# INPUT:  A - np.ndarray (2D), d - float (from 0.0 to 1.0), x - np.ndarray (1D, size of A.shape[0/1])
# OUTPUT: y - np.ndarray (1D, size of x)
def pagerank_matvec(A, d, x): # 2 pts
    # enter your code here
    return y

#### DBLP: computer science bibliography

Download the dataset from [here](https://goo.gl/oZVxEa), unzip it and put `dblp_authors.npz`  and `dblp_graph.npz` in the same folder with this notebook. Each value (author name) from `dblp_authors.npz` corresponds to the row/column of the matrix from `dblp_graph.npz`. Value at row `i` and column `j` of the matrix from `dblp_graph.npz` corresponds to the number of times author `i` cited papers of the author `j`. Let us now find the most significant scientists according to PageRank model over DBLP data.

* (4 pts) Load the weighted adjacency matrix and the authors list into Python using ```load_dblp(...)``` function. Print its density (fraction of nonzero elements). Find top-10 most cited authors from the weighted adjacency matrix. Now, make all the weights of the adjacency matrix equal to 1 for simplicity (consider only existence of connection between authors, not its weight). Obtain the PageRank matrix $A$ from the adjacency matrix and verify that it is stochastic.
 
 
* (1 pts) In order to provide ```pagerank_matvec``` to your ```power_method``` (without rewriting it) for fast calculation of $A_dx$, you can create a ```LinearOperator```: 
```python
L = scipy.sparse.linalg.LinearOperator(A.shape, matvec=lambda x, A=A, d=d: pagerank_matvec(A, d, x))
```
Calling ```L@x``` or ```L.dot(x)``` will result in calculation of ```pagerank_matvec(A, d, x)``` and, thus, you can plug $L$ instead of the matrix $A$ in the ```power_method``` directly. **Note:** though in the previous subtask graph was very small (so you could disparage fast matvec implementation), here it is very large (but sparse), so that direct evaluation of $A_dx$ will require $\sim 10^{12}$ matrix elements to store - good luck with that (^_<).


* (2 pts) Run the power method starting from the vector of all ones and plot residuals $\|A_dx_k - λ_k x_k\|_2$  as a function of $k$ for $d=0.85$.


* (1 pts) Print names of the top-10 authors according to PageRank over DBLP when $d=0.85$. Comment on your findings.

In [ ]:
from scipy.sparse import load_npz
import numpy as np
def load_dblp(path_auth, path_graph):
    G = load_npz(path_graph).astype(float)
    with np.load(path_auth) as data: authors = data['authors']
    return G, authors
G, authors = load_dblp('dblp_authors.npz', 'dblp_graph.npz')

In [ ]:
# Your code is here

## Problem 3. QR algorithm (33 pts)

* Implement QR-algorithm without shifts. Prototype of the function is given below

In [ ]:
# INPUT: 
# A_init - square matrix, 
# num_iter - number of iterations for QR algorithm
# OUTPUT: 
# Ak - transformed matrix A_init given by QR algorithm, 
# convergence - numpy array of shape (num_iter, ), 
# where we store the maximal number from the Chebyshev norm 
# of triangular part of the Ak for every iteration
def qr_algorithm(A_init, num_iter): # 3 pts
    # enter your code here
    return Ak, convergence

#### Symmetric case (3 pts)
- Create symmetric tridiagonal $11 \times 11$ matrix with elements $-1, 2, -1$ on sub-, main- and upper diagonal respectively without using loops.
- Run $400$ iterations of the QR algorithm for this matrix.
- Plot the output matrix with function ```plt.spy(Ak, precision=1e-7)```.
- Plot convergence of QR-algorithm.

In [ ]:
# Your solution is here

#### Nonsymmetric case (5 pts)

- Create nonsymmetric tridiagonal $11 \times 11$ matrix with elements $5, 3, -2$ on sub-, main- and upper diagonal respectively without using loops.
- Run $250$ iterations of the QR algorithm for this matrix.
- Plot the result matrix with function ```plt.spy(Ak, precision=1e-7)```. Is this matrix lower triangular? How does this correspond to the claim about convergence of the QR algorithm?

In [ ]:
# Your solution is here

### QR algorithms with Rayleigh Quotient shift (10 pts)

In the lectures the Rayleigh Quotient shift was introduced to speed up convergence of power method. Here we ask you to generalize this approach to construct the shifts in QR algorithm.

- How to compute the Rayleigh Quotient shift in QR algorithm fast? Provide formulas and explanations how they can be simplified.
- Implement explicit QR algorithm with Rayleigh Quotient shift. Please do not worry about implicit orthogonalization, we want to compare convergence only in terms of iterations.
- Test your implementation in the symmetric case. Plot the convergence of QR algorithm with and without shift. Choose the dimension $n \sim 100 $ for more representative results. 
- How the convergence of the shifted algorithm compares to the simple QR? Why? 

In [ ]:
def qr_algorithm_reileigh(A_init, num_iter):
    # enter your code here
    return Ak, convergence

- Try QR with Rayleigh Quotient shift for a simple matrix $A = \begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}$. Does anything change from iteration to iteration? Does shift affect convergence here? Why?

In [ ]:
# Your solution is here

### QR with Wilkinson shift  (15 pts)

To solve the problem that appears in the last example, we can use the Wilkinson shift:

$$\mu = a_m - \frac {sign(\delta) b^2_{m-1}} {(|\delta| + \sqrt{\delta^2 + b^2_{m-1}} )},$$

where $\delta = \frac{(a_{m-1} - a_m)}{2}$. If $\delta = 0$, then instead of $sign(\delta)$ you have to choose $1$ or $-1$ arbitrary.
The numbers $a_m, b_{m-1}, a_{m-1}$ are taken from matrix $B$:

$$
    B = 
    \begin{bmatrix} 
     a_{m-1} & b_{m-1} \\ 
     b_{m-1} & a_m \\ 
    \end{bmatrix},
$$  
which is a lower right bottom submatrix of $A^{(k)}$. Here $k$ is an iteration counter in QR algorithm.

- Compare convergence in the symmetric cases: 
    - distinctive eigenvalues
    - two coincident eigenvalues
    - maximum and minimum eigenvalues with the same absolute value
Choose the dimension $n \sim 100 $ for more representative results.
What do you observe? 

In [ ]:
def qr_algorithm_wilkinson(A_init, num_iter):
    # enter your code here
    return Ak, convergence

In [ ]:
# Your solution is here

## Problem 4. (Movie Recommender system) 15 pts

Imagine the world without NLA where you have free evenings and you can watch movies!  
But it is always hard to choose a movie to watch. 
In this problem we suggest you to build your own movie recommender system based on SVD decomposition, so you can combine two perfect things: Numerical Linear Algebra and cinematography!

In order to build recommender system you need data. 
Here you are https://grouplens.org/datasets/movielens/1m/

Usually all recommender systems may be devided into two groups

#### Collaborative filtering. 

This approach is based on user-item interaction.
It has one important assumption: user who has liked an item in the past will also likes the same in the future. Suppose the user A likes the films about vampires. 
He is Twilight saga fan and he has watched the film "What we do in the shadows" and liked it or unliked it, in other words he evaluated it somehow. And suppose another user B, who has the similair behavior to the first user (he is also Twilight saga fan). And the chance, that he will estimate "What we do in the shadows" in the same way that user A did, is huge. So, the purpose of the collaborative filtering is to predict a user's behavior based on behavior of the simular users.

#### Content based filtering.

Collaborative filtering has some essential flaws. The main one is called "cold start". "Cold start" happens when the new user comes and he has not react anyhow to the items. So we do not know his past behavior and we do not know what to advise. Here content based filtering helps. Often resources gather some extra info about users and items before a user comes down to utilising the resource. So, for example we would know that user likes horror movies before he watched anything on the resource.


- In this task you will implement Collaborative filtering based on SVD (we will use the function from the proper package and check if the result recommender system advices the similar movies)

1) (1 pts)  Explore the data. Construct the interaction matrix $M$ of size $m \times n$ which contains the information of how a certain user rated a certain film. 

2) (5 pts)  Compute SVD of this matrix. Remeber that matrix $M$ is sparse (one user can hardly watch all the movies) so the good choice would be to use method from ```scipy.sparse.linalg``` package

$$ M = USV^{\top}, $$

where $U$ is a $m \times r $ orthogonal matrix with left singular vectors, which represents the relationship between users and latent factors, $S$ is a $r \times r $ diagonal matrix, which describes the strength of each latent factor and $V^\top$ is a $r \times n$ matrix with right singular vectors, which represent the embeddings of  items (movies in our case) in latent space.
Describe any simple heuristic to choose appropriate value for $r$ and explain why do you expect that it will work.


In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd

# Read the dataset

# Create the interaction matrix

# Normalize the matrix


In [ ]:
# Compute Singular Value Decomposition of interaction matrix. You can use built-in functions

U,S, V = svd(M) # Update this line, it is just example

3) (2 pts) In order to get weighted item-latent factors, we can multiply $S$ and $V^{T}$. Please, remember that $S$ is diagonal and multiply them efficiently.

In [ ]:
# Your solutuion is here

Now we have vectors that represent our item space. In other words we have $N$ movies and $N$ vectors which describe each movie, a.k.a. embeddings. 
In order to know if two movies are similar or not we need just to check if the corresponding vectors are similair or not. How we can do this?

4) (2 pts)  Implement the cosine metric. If the cosine metric between two vectors equals to $1$ both vectors are collinear, if $0$ vectors are orthogonal, as a result corresponding movies are completely different.

$$
cosine(u,v) = \frac{u^{\top}v}{\|u\|_2\|v\|_2}
$$

In [ ]:
# Your solutuion is here

5) (5 pts) Check your result. Implement the fuction, which finds and prints $k$ similar movies to the one you have chosen

In [ ]:
# Your solutuion is here

Enjoy watching the recommended movies!
